# 기후자료 통계분석 코드 분석 과제
> nino

- toc: true
- branch: master
- badges: true
- comments: true
- author: 김혜지
- categories: [python]

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
#라이브러리 불러오기
dset = xr.open_dataset(‘./Data/sst.mnmean.nc’) #데이터 셋 불러오기.
sst=dset.sst.sel() #데이터 셋 중 sst를 선택해서 sst라고 저장
nino34 = sst.where((sst.lat<5) & (sst.lat>-5) & (sst.lon>190) & (sst.lon<240),
drop=True).mean(dim=['lat','lon']) #sst 중 위도가 -5에서 5사이 그리고 경도가 190에서 240 사이에 위치하는 데이터 셋을 고르기, 이 조건을 만족하지 못하는 레이블은 결과에서 삭제하고, 경도와 위도를 평균으로 하는 데이터 셋을 nino34에 저장

clim = nino34.sel(time=slice(‘1982-01',’2018-12')).groupby('time.month').mean(dim='time') #시간을 선택해서 달별로 그룹화, 시간대상 평균을 내고 clim에 저장
anom = (nino34.groupby('time.month') - clim).sel(time=slice(‘1981-12','2019-02')) #각 달만큼 그룹화 한 것 - 달마다 평균낸 것 -> 편차를 atom에 저장
 
nino3mon = anom.rolling(time=5, center=True).mean() #5개월 단위로(롤링)평균 구해서 nino3mon에 저장
ninos = nino3mon[12::12] #달 선택

#그림판 그리기 
fig = plt.figure(figsize = (8,6), dpi = 150) 
ax = fig.subplots()

#그림 꾸미기
ax.plot(ninos.time.values, ninos, ‘b-') 
ax.axhline(0,color='black',linewidth=0.5) #
ax.axhline(1.0,color='black',linewidth=0.5,linestyle='dashed')
ax.axhline(-1.0,color='black',linewidth=0.5,linestyle='dashed')
ax.axhline(1.5,color='black',linewidth=0.6,linestyle='dashed')
ax.axhline(-1.5,color='black',linewidth=0.6,linestyle='dashed')
ax.axhline(2.0,color='black',linewidth=0.7,linestyle='dashed')
ax.axhline(-2.0,color='black',linewidth=0.7,linestyle='dashed’)
ax.axhline(ninos.std().values,color='red',linewidth=0.5,linestyle='dashed')
#x축 y축 이름 설정
ax.set_xlabel('Time')
ax.set_ylabel('[$\degree$C]')
#타이틀 설정하기
ax.set_title('Nino 3.4 index') 
#이미지 저장
plt.savefig('nino34_index_season.jpg’) 
plt.show()